Через данную программу получалась итоговая посылка на private test. Вычисления проводились в google colab. Для получения той же посылки достаточно поочередно активировать все блоки кода и заменить адреса файлов на ссылки/адреса которые стоят у вас. я подключал прямо с диска потому и включаю с диска. Если по каким-то причинам результаты отличаются или модель не работает, ну или вам не нравится такое решение, прошу написать мне в telegram @PavelGladkih

Замечание! Я опасаюсь, что при сжатии/загрузке на github train.csv потерялись какие-то данные, так что его по возможности лучше подгрузить его как-то иначе!!! ( не с гитхаб)

In [1]:
import numpy as np
import pandas as pd

# Эти 3 строки подразумевают загрузку обучающего набора train.csv и private_test
from google.colab import drive
drive.mount('/content/private_test.csv')
drive.mount('/content/train.csv')
drive.mount('/content/sample_submission.csv')

Mounted at /content/private_test.csv
Mounted at /content/train.csv
Mounted at /content/sample_submission.csv


Загрузка обучающей выборки и построение по ней начального словаря с сохранением частот встречи с каждым словом (начального т.к. в нем много слов с разного рода опечатками и их нужно убирать). Также загружается словарь с большим известным числом русских слов в разных формах. по нему строится новый словарь, который впоследствии будет применяться для улучшения начального словаря

In [2]:
train = pd.read_csv('/content/sample_submission.csv/MyDrive/train.csv')

import re
set(re.sub('[!@#$-,.?]', '', train['correct_text'][0]).split(  ))

train_correct = train['correct_text']


lists = list()
buf_Dictionary = dict()
i=0
for string in train_correct:
  new_list = re.sub('[1234567890qwertyuiopasdfghjklzxcvbnm!@#$--,.?¬\]\[\'\"]', ' ', string.lower()).split(  )
  for i in new_list:
    if i in buf_Dictionary:
      buf_Dictionary[i] = buf_Dictionary[i] + 1
    else:
      buf_Dictionary[i] = 1


import requests

response = requests.get('https://raw.githubusercontent.com/danakt/russian-words/master/russian.txt')

text = response.content.decode('cp1251')

with open('russian.txt', 'wb') as ru:
    ru.write(text.encode('utf-8'))

f = open('russian.txt', 'r')

Dict_FULL = dict()
with open('russian.txt', 'r')  as f:
  nums = f.read().splitlines()
for i in nums:
  if i[0]=='-':
    Dict_FULL[i[1:]] = 1
  else:
    Dict_FULL[i] = 1

def perc(crit):
  not_in = 0
  in_dict = 0
  critical = crit
  for i in buf_Dictionary:
    if i in Dict_FULL and buf_Dictionary[i]<critical:
      in_dict = in_dict + 1
    if i not in Dict_FULL and buf_Dictionary[i]<critical:
      not_in = not_in +1
  return not_in, not_in/float(in_dict+not_in)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: Possible set difference at position 41
  del sys.path[0]


perc функция которая подсчитывает статистики для анализа, какие слова можно выкинуть. 

Следующий блок задает основной словарь. Используется словарь изначальный и из него выбрасываются слова, если они не встречаются в более полном словаре из интернета и редко встречаются ( второй if удаляет множество лишних слов длиной меньше 4 и для удобства добавляется элемент с пустой строкой)

In [3]:
Dictionary = buf_Dictionary.copy()
print(len(Dictionary))
critical = 5
print(perc(critical))
Dictionary = buf_Dictionary.copy()
for i in buf_Dictionary:
  if i not in Dict_FULL and buf_Dictionary[i]<critical:
    Dictionary.pop(i)
  else:
    if ((len(i)<4) and ((buf_Dictionary[i]<50) and (i not in Dict_FULL))):
      Dictionary.pop(i)


Dictionary[''] = 1
print(len(Dictionary))

266051
(89123, 0.45712541802589196)
176413


In [4]:
!pip install python-Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 5.3 MB/s 
     |████████████████████████████████| 2.2 MB 46.9 MB/s 


Сама модель. она состоит из 4 функций. 

In [5]:
import Levenshtein



sonority = dict()
 
sonority['б'] = 'п'
sonority['п'] = 'б'

sonority['в'] = 'ф'
sonority['ф'] = 'в'

sonority['г'] = 'к'
sonority['к'] = 'г'

sonority['д'] = 'т'
sonority['т'] = 'д'

sonority['ж'] = 'ш'
sonority['ш'] = 'ж'

sonority['з'] = 'с'
sonority['с'] = 'з'

sonority['а'] = 'я'
sonority['я'] = 'а'

sonority['у'] = 'ю'
sonority['ю'] = 'у'

sonority['э'] = 'е'
sonority['е'] = 'э'

sonority['ы'] = 'и'
sonority['и'] = 'ы'

sonority['о'] = 'ё'
sonority['ё'] = 'о'



def checksonority(word):
  for i in range(len(word)):
    if word[i] == 'о':
      newword = word[:i] + 'а' + word[i+1:]
      if newword in Dictionary:
        return newword

    if word[i] == 'а':
      newword = word[:i] + 'о' + word[i+1:]
      if newword in Dictionary:
        return newword
      
      if word[i] in sonority:
        newword = word[:i] + sonority[word[i]] + word[i+1:]
        if newword in Dictionary:
          return newword
  return ''

def checkperm(word):

  for i in range(len(word)-1):
    newword = word[:i]+ word[i+1] + word[i] + word[i+2:]
    if newword in Dictionary:
      return newword
  return ''




def find_zam(word):
  distance_words = 150


  replacement = checksonority(word)
  if replacement != '':
    return replacement


  replacement = checkperm(word)
  if replacement != '':
    return replacement


  for dictory_word in Dictionary:
     if (abs(len(dictory_word)- len(word))<3):
       new_dist = Levenshtein.distance(word,dictory_word)
       if (new_dist<distance_words):
         replacement = dictory_word
         distance_words = new_dist
       else:
         if (new_dist-distance_words<1) and (Dictionary[replacement]<Dictionary[dictory_word]):
           replacement = dictory_word
           distance_words = new_dist
  return replacement




russianlatters = 'абвгдежзийклмнопрстуфхцчшщъыьэюя' + 'абвгдежзийклмнопрстуфхцчшщъыьэюя'.upper()


def sentence_correction(sentense):
  newsentense = sentense
  for i in range(0, len(sentense)-1):
    if sentense[i] =='a':
      if (i-1>=0 and sentense[i-1] in russianlatters) or (i+1<=len(sentense)-1 and sentense[i+1] in russianlatters):
        newsentense = sentense[:i] + 'а' + sentense[i+1:]

    if sentense[i] =='A':
      if (i-1>=0 and sentense[i-1] in russianlatters) or (i+1<=len(sentense)-1 and sentense[i+1] in russianlatters):
        newsentense = sentense[:i] + 'А' + sentense[i+1:]



    if sentense[i] =='c':
      if (i-1>=0 and sentense[i-1] in russianlatters) or (i+1<=len(sentense)-1 and sentense[i+1] in russianlatters):
        newsentense = sentense[:i] + 'с' + sentense[i+1:]

    if sentense[i] =='C':
      if (i-1>=0 and sentense[i-1] in russianlatters) or (i+1<=len(sentense)-1 and sentense[i+1] in russianlatters):
        newsentense = sentense[:i] + 'С' + sentense[i+1:]
    
    if sentense[i] =='o':
      if (i-1>=0 and sentense[i-1] in russianlatters) or (i+1<=len(sentense)-1 and sentense[i+1] in russianlatters):
        newsentense = sentense[:i] + 'о' + sentense[i+1:]
    if sentense[i] =='O':
      if (i-1>=0 and sentense[i-1] in russianlatters) or (i+1<=len(sentense)-1 and sentense[i+1] in russianlatters):
        newsentense = sentense[:i] + 'О' + sentense[i+1:]


    if sentense[i] =='B':
      if (i-1>=0 and sentense[i-1] in russianlatters) or (i+1<=len(sentense)-1 and sentense[i+1] in russianlatters):
        newsentense = sentense[:i] + 'В' + sentense[i+1:]

    if sentense[i] =='T':
      if (i-1>=0 and sentense[i-1] in russianlatters) or (i+1<=len(sentense)-1 and sentense[i+1] in russianlatters):
        newsentense = sentense[:i] + 'Т' + sentense[i+1:]

  sentense = newsentense
  wordslist = re.sub('[1234567890qwertyuiopasdfghjklzxcvbnm:!@#$--,.?\]\[\"\']', ' ', sentense.lower()).split()
  notlowerlist = re.sub('[1234567890qwertyuiopasdfghjklzxcvbnmQWERTYUIOPASDFGHJKLZXCVBNM:!@#$--,.?\]\[\"\']', ' ', sentense).split()
  j = 0
  buf = ''
  if len(wordslist)!=len(notlowerlist):
    print(sentense)
  for i in wordslist:
    if i not in Dictionary:
      pattern = r'\b'+ notlowerlist[j] + r'\b'
      if notlowerlist[j].istitle():
        sentense = re.sub(pattern, find_zam(i).title(), sentense) #sentense.replace(notlowerlist[j],find_zam(i).title())
      else:
        sentense = re.sub(pattern, find_zam(i), sentense) #sentense.replace(notlowerlist[j],find_zam(i))
    j = j + 1
  return sentense

загрузка test'а

In [6]:
test = pd.read_csv('/content/sample_submission.csv/MyDrive/private_test.csv')

Функция для обработки текста по частям (применялась для удобства и ускорения работы на публичном и приватном датасете)

In [7]:
blocksize = 5454 

new_test = test.copy()

def new_test_data(index):
  if index < 11:
    new_test['corrupted_text'][((index-1)*blocksize):(index*blocksize)] = test['corrupted_text'][((index-1)*blocksize):(index*blocksize)].apply(sentence_correction)
  if index == 11:
    new_test['corrupted_text'][((index-1)*blocksize):] = test['corrupted_text'][((index-1)*blocksize):].apply(sentence_correction)
  new_test.to_csv('_2215_.csv', index= False, header=False)
  new_test.to_csv('2215_.csv', index= False, header=True)

In [ ]:
for i in range(1,12):
  print(i)
  new_test_data(i)

1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:150: FutureWarning: Possible set difference at position 42
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:151: FutureWarning: Possible set difference at position 68


2
3
4
5
6
7
8
9
10
11


In [ ]:
new_test.sample(15)

,corrupted_text
37641,Конструктор Roblox коллекция сюрприз серия
11155,Можешь снова повторить Эту речь?
2814,"Не пережила, оставь это мне."
19512,Позвоните ему. Сейчас же.
10717,"Назовете, как хотите."
52923,А как ты нашла дорогу?
21530,Некий человек...
52935,Вы работа три часа?
52001,"Да, конечно. Я слышала... Вы здесь бывал раньше."
48297,У меня были хорхе оценки по этикет и гигиене.


Осталось сохранить решение

In [ ]:
new_test.to_csv('_FF2.csv', index= False, header=False)

In [ ]:
pd.read_csv('/content/_FF2.csv')

,мясные блюда говядина
0,- А можно я пойд?
1,Бордюр обойн ассортименте
2,Вместо соуса кетчуп
3,"Не простая, как она могла туда папа."
4,У мня таких полное поле.
...,...
56520,Замена втулок стабилизатора задних
56521,Пряники Воронежские ржаные
56522,"Следовательно, он вновь."
56523,Детская стрижка дошкольный возраст


Для проверки совпадают ли решения загрузим финальную посылку, посчитаем некоторые строки и сравним их (последнюю посылку загрузил руками)

In [20]:
Final_sub = pd.read_csv('/content/_FF2.csv',header=None)

In [21]:
Final_sub

,0
0,мясные блюда говядина
1,- А можно я пойд?
2,Бордюр обойн ассортименте
3,Вместо соуса кетчуп
4,"Не простая, как она могла туда папа."
...,...
56521,Замена втулок стабилизатора задних
56522,Пряники Воронежские ржаные
56523,"Следовательно, он вновь."
56524,Детская стрижка дошкольный возраст


In [22]:
smp = test.sample(15)

In [23]:
smp['corrupted_text'].apply(sentence_correction)

2169                   Лента траурная декоративная
13491                   Ну что мне с тобой делать?
7768     - Это так не похоже на тебя, дядя Скрудж.
20337                    Ну же Чак, возьми трубку.
29524                     Позволь ему рыть самцом.
33552     Рано или поздно так людей учит в обмане.
16649                       - Топ я тебе и сказал.
32280      К концу дня один останется, один умрет.
13685        Знате чао? Я хочу установить правило.
46662                          -Мы там вместе был.
1716               какой у ва любимый парк? Линки.
13664                            И это невозможно.
50187     На побережье до сих пор ничего не нашли.
12921              Мы с Этаном однажды беседовали.
15232         Ручка масляная сер DignoTrijet синяя
Name: corrupted_text, dtype: object

In [24]:
Final_sub[0].iloc[smp.index]

2169                   Лента траурная декоративная
13491                   Ну что мне с тобой делать?
7768     - Это так не похоже на тебя, дядя Скрудж.
20337                    Ну же Чак, возьми трубку.
29524                     Позволь ему рыть самцом.
33552     Рано или поздно так людей учит в обмане.
16649                       - Топ я тебе и сказал.
32280      К концу дня один останется, один умрет.
13685        Знате чао? Я хочу установить правило.
46662                          -Мы там вместе был.
1716               какой у ва любимый парк? Линки.
13664                            И это невозможно.
50187     На побережье до сих пор ничего не нашли.
12921              Мы с Этаном однажды беседовали.
15232         Ручка масляная сер DignoTrijet синяя
Name: 0, dtype: object

Как можно увидеть результаты совпали, что говорит о воспроизводимости. Если у вас по каким-то причинам они отличаются или модель не работает, ну или вам не нравится такое решение, прошу написать мне в telegram @PavelGladkih